In [1]:
import pandas as pd
import numpy as np
import random
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib
import warnings
import pickle
import geopandas as gpd
import os

# matplotlib.style.use('ggplot')
# plt.rcParams['axes.facecolor']='w'
warnings.filterwarnings('ignore')

import time

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.utils import class_weight
from sklearn.model_selection import KFold


In [ ]:
# import sys
# import argparse
# parser = argparse.ArgumentParser(
#                     prog='Test Multiple Model',
#                     description='For Classification & Hurdle tasks, test the model performace',
#                     epilog='')
# parser.add_argument('-sp', '--species')      # option that takes a value
# parser.add_argument('-year','--year')
# parser.add_argument('-size', '--sample_size')  # on/off flag
# parser.add_argument('-o','--output_path', default='./Model_Performance/')
# args = parser.parse_args()
# print(args)

# sp = args.species
# year = int(args.year)
# SAMPLE_SIZE = int(args.sample_size)
# OUTPUT_DIR = os.path.join(args.output_path, sp)

# if not os.path.exists(OUTPUT_DIR):
#     os.makedirs(OUTPUT_DIR)

In [2]:
sp = 'Mallard'
year = 2020
SAMPLE_SIZE = 1000
OUTPUT_DIR = f'./Model_Performance/{sp}'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from BirdSTEM.utils.plot_gif import make_sample_gif, make_sample_gif_hexagon


# load training data


In [5]:
checklist_data = pd.read_csv(f'./BirdSTEM/dataset/test_data/checklist_data/checklist_data_filtered_{year}.csv')

### mallard 2020
with open(f'./BirdSTEM/dataset/test_data/sp_data/{sp}/{sp}_{year}.pkl','rb') as f:
    sp_data = pickle.load(f)
    
checklist_data = checklist_data.merge(sp_data, on='sampling_event_identifier', how='left')
checklist_data['count'] = checklist_data['count'].fillna(0)


# Train test split

In [6]:
from sklearn.model_selection import train_test_split
from BirdSTEM.dataset.get_test_x_names import get_test_x_names

x_names = get_test_x_names()
X = checklist_data[['sampling_event_identifier','longitude','latitude'] + x_names]
y = checklist_data['count'].values

_, X, _, y = train_test_split(X, y, test_size=SAMPLE_SIZE, stratify=np.where(y>0, 1, 0))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3,
                                                    stratify = np.where(y>0, 1, 0), shuffle=True)




# Test model

## Task1: Classification (modeling occurrence)

### First, without AdaSTEM wrapper

In [7]:
from BirdSTEM.model.AdaSTEM import AdaSTEM, AdaSTEMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from elapid import MaxentModel


In [8]:

cls_model_set_dict = {
    'LogisticRegression':LogisticRegression(random_state=42),
    'SGDClassifier':SGDClassifier(random_state=42),
    'GaussianNB':GaussianNB(),
    'KNeighborsClassifier':KNeighborsClassifier(),
    'DecisionTreeClassifier':DecisionTreeClassifier(random_state=42),
    # 'SVC_linear':SVC(kernel='linear',random_state=42),
    'SVC_rbf':SVC(kernel='rbf',random_state=42),
    'RandomForestClassifier':RandomForestClassifier(random_state=42),
    'QuadraticDiscriminantAnalysis':QuadraticDiscriminantAnalysis(),
    'MLPClassifier':MLPClassifier(random_state=42),
    'MaxEnt':MaxentModel(transform='cloglog', beta_multiplier=2.0),
    'XGBClassifier':XGBClassifier(tree_method='hist',random_state=42, n_jobs=1),
    'ComplementNB':ComplementNB(),
}


In [9]:
K=5

cls_metric_df_list = []
for item in list(cls_model_set_dict.keys())[::-1]:
    model_name = item
    model = cls_model_set_dict[model_name]
    
    kf = KFold(n_splits=K, shuffle=True, random_state=42).split(X, y)
    for kf_count, (train_index, test_index) in tqdm(enumerate(kf), desc=f'{model_name}', total=K):
        
        try:
            X_train = X.iloc[train_index].replace(-1,np.nan)
            imputer = SimpleImputer().fit(X_train[x_names])
            X_train[x_names] = imputer.transform(X_train[x_names])
            scaler = MinMaxScaler().fit(X_train[x_names])
            X_train[x_names] = scaler.transform(X_train[x_names])
            
            y_train = np.where(y[train_index]>0, 1, 0)
            
            X_test = X.iloc[test_index].replace(-1,np.nan)
            X_test[x_names] = imputer.transform(X_test[x_names])
            X_test[x_names] = scaler.transform(X_test[x_names])
            y_test = np.where(y[test_index]>0, 1, 0)
            
            
            sample_weights = class_weight.compute_sample_weight(class_weight='balanced',y=y_train)
            
            a = time.time()

            try:
                start_time = time.time()
                model.fit(X_train[x_names], y_train, sample_weight=sample_weights)
                finish_time = time.time()
                training_time = finish_time - start_time
            except:
                start_time = time.time()
                model.fit(X_train[x_names], y_train)
                finish_time = time.time()
                training_time = finish_time - start_time
                
            start_time = time.time()
            y_pred = model.predict(X_test[x_names])
            finish_time = time.time()
            predicting_time = finish_time - start_time
            
            y_pred = np.where(y_pred<0, 0, y_pred)
            metric_df = AdaSTEM.eval_STEM_res('classification', y_test, y_pred)
            
            metric_df['model'] = model_name
            metric_df['task_type'] = 'classification'
            metric_df['iter'] = kf_count
            metric_df['sp'] = sp
            metric_df['sample_size'] = SAMPLE_SIZE
            metric_df['training_time'] = training_time
            metric_df['predicting_time'] = predicting_time
            
            cls_metric_df_list.append(metric_df)
            
            print(metric_df)
            
        except Exception as e:
            print(e)
            continue



    

ComplementNB:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.5844674287788061, 'kappa': 0.09768125065575495, 'f1': 0.3174603174603175, 'precision': 0.21505376344086022, 'recall': 0.6060606060606061, 'average_precision': 0.19533561420658196, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'ComplementNB', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.004089832305908203, 'predicting_time': 0.002395153045654297}
{'AUC': 0.6283641916396259, 'kappa': 0.13001449975833723, 'f1': 0.338235294117647, 'precision': 0.21904761904761905, 'recall': 0.7419354838709677, 'average_precision': 0.20251920122887865, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'ComplementNB', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.0030660629272460938, 'predicting_time': 0.002087831497192383}
{'AUC': 0.5386499727817093, 'kappa':

XGBClassifier:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.5215024496461621, 'kappa': 0.05870696061431746, 'f1': 0.13636363636363635, 'precision': 0.2727272727272727, 'recall': 0.09090909090909091, 'average_precision': 0.17479338842975206, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'XGBClassifier', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.1596369743347168, 'predicting_time': 0.004869937896728516}
{'AUC': 0.5566902080549723, 'kappa': 0.12137310993052719, 'f1': 0.2456140350877193, 'precision': 0.2692307692307692, 'recall': 0.22580645161290322, 'average_precision': 0.1807940446650124, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'XGBClassifier', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.16182827949523926, 'predicting_time': 0.004032135009765625}
{'AUC': 0.5910905461803666, 'kappa': 

MaxEnt:   0%|          | 0/5 [00:00<?, ?it/s]

...

{'AUC': 0.5567047722736346, 'kappa': 0.04159733777038266, 'f1': 0.3076923076923077, 'precision': 0.18285714285714286, 'recall': 0.9696969696969697, 'average_precision': 0.18231601731601732, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'GaussianNB', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.0041637420654296875, 'predicting_time': 0.0022220611572265625}
{'AUC': 0.5564993319335751, 'kappa': 0.040294037571467545, 'f1': 0.29145728643216084, 'precision': 0.17261904761904762, 'recall': 0.9354838709677419, 'average_precision': 0.17148233486943165, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'GaussianNB', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.0038051605224609375, 'predicting_time': 0.0022258758544921875}
{'AUC': 0.5077118490292143, 'kappa

SGDClassifier:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.6785519869352206, 'kappa': 0.19166342033502148, 'f1': 0.39416058394160586, 'precision': 0.25961538461538464, 'recall': 0.8181818181818182, 'average_precision': 0.24241258741258745, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'SGDClassifier', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.006205081939697266, 'predicting_time': 0.0032320022583007812}
{'AUC': 0.566711204428326, 'kappa': 0.0469159003959998, 'f1': 0.29702970297029696, 'precision': 0.17543859649122806, 'recall': 0.967741935483871, 'average_precision': 0.1747792869269949, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'SGDClassifier', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.017700672149658203, 'predicting_time': 0.0029799938201904297}
{'AUC': 0.5489929232444203, 'kappa

LogisticRegression:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.6750136091453457, 'kappa': 0.21130463358527762, 'f1': 0.4, 'precision': 0.27586206896551724, 'recall': 0.7272727272727273, 'average_precision': 0.24562695924764888, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'LogisticRegression', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.020888090133666992, 'predicting_time': 0.002315998077392578}
{'AUC': 0.6888719221225424, 'kappa': 0.21100330525642386, 'f1': 0.3934426229508197, 'precision': 0.26373626373626374, 'recall': 0.7741935483870968, 'average_precision': 0.23918291386033322, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'LogisticRegression', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.032016754150390625, 'predicting_time': 0.0022771358489990234}
{'AUC': 0.5927236436218474, 'kappa': 0

In [10]:
cls_metric_df = pd.DataFrame(cls_metric_df_list)
cls_metric_df.to_csv(os.path.join(OUTPUT_DIR, f'cls_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv'),
                                        index=False)


### Then, with AdaSTEM wrapper

In [11]:

K=5

Ada_cls_metric_df_list = []
for item in list(cls_model_set_dict.keys())[::-1]:
    model_name = item
    
    kf = KFold(n_splits=K, shuffle=True, random_state=42).split(X, y)
    for kf_count, (train_index, test_index) in tqdm(enumerate(kf), desc=f'AdaSTEM + {model_name}', total=K):
        
        try:
            X_train = X.iloc[train_index].replace(-1,np.nan)
            
            new_x_names = list(set(x_names) - set(['DOY']))
            
            imputer = SimpleImputer().fit(X_train[new_x_names])
            X_train[new_x_names] = imputer.transform(X_train[new_x_names])
            scaler = MinMaxScaler().fit(X_train[new_x_names])
            X_train[new_x_names] = scaler.transform(X_train[new_x_names])
            
            y_train = np.where(y[train_index]>0, 1, 0)
            X_test = X.iloc[test_index].replace(-1,np.nan)
            X_test[new_x_names] = imputer.transform(X_test[new_x_names])
            X_test[new_x_names] = scaler.transform(X_test[new_x_names])
            y_test = np.where(y[test_index]>0, 1, 0)
            
            model = AdaSTEMClassifier(base_model=cls_model_set_dict[model_name], 
                                        sample_weights_for_classifier=True,
                                        ensemble_fold=5,
                                        min_ensemble_require=3,
                                        grid_len_lon_upper_threshold=25,
                                        grid_len_lon_lower_threshold=5,
                                        grid_len_lat_upper_threshold=25,
                                        grid_len_lat_lower_threshold=5,
                                        points_lower_threshold=50,
                                        stixel_training_size_threshold=50,
                                        temporal_start = 1,
                                        temporal_end = 367, 
                                        temporal_step = 30.5, 
                                        temporal_bin_interval = 30.5,
                                        save_tmp=False, 
                                        save_gridding_plot=False)
            
            try:
                start_time = time.time()
                model.fit(X_train[new_x_names + ['DOY','longitude', 'latitude']], y_train)
                finish_time = time.time()
                training_time = finish_time - start_time
            except:
                start_time = time.time()
                model.set_params(**{'sample_weights_for_classifier':False})
                model.fit(X_train[new_x_names + ['DOY','longitude', 'latitude']], y_train)
                finish_time = time.time()
                training_time = finish_time - start_time
                
            start_time = time.time()
            y_pred = model.predict(X_test[new_x_names + ['DOY','longitude', 'latitude']])
            finish_time = time.time()
            predicting_time = finish_time - start_time
            
            y_pred = np.where(y_pred<0, 0, y_pred)
            metric_df = AdaSTEM.eval_STEM_res('classification', y_test, y_pred)
            
            metric_df['model'] = 'AdaSTEM_' + model_name
            metric_df['task_type'] = 'classification'
            metric_df['iter'] = kf_count
            metric_df['sp'] = sp
            metric_df['sample_size'] = SAMPLE_SIZE
            metric_df['training_time'] = training_time
            metric_df['predicting_time'] = predicting_time
        
            Ada_cls_metric_df_list.append(metric_df)
            print(metric_df)
            
        except Exception as e:
            print(e)
            continue



    

AdaSTEM + ComplementNB:   0%|          | 0/5 [00:00<?, ?it/s]

training: 100%|██████████| 1231/1231 [00:00<00:00, 9645.45it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.165, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_ComplementNB', 'task_type': 'classification', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.3953828811645508, 'predicting_time': 0.720020055770874}


training: 100%|██████████| 1170/1170 [00:00<00:00, 9625.34it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.155, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_ComplementNB', 'task_type': 'classification', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.3399977684020996, 'predicting_time': 0.7334439754486084}


...

{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.165, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_LogisticRegression', 'task_type': 'classification', 'iter': 2, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.34155988693237305, 'predicting_time': 0.7393488883972168}


training: 100%|██████████| 1206/1206 [00:00<00:00, 9892.07it/s] 


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.155, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_LogisticRegression', 'task_type': 'classification', 'iter': 3, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.3452491760253906, 'predicting_time': 0.7037067413330078}


training: 100%|██████████| 1245/1245 [00:00<00:00, 9639.07it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.205, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': nan, 'MAE': nan, 'MSE': nan, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_LogisticRegression', 'task_type': 'classification', 'iter': 4, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.39054298400878906, 'predicting_time': 0.7296769618988037}


In [12]:
Ada_cls_metric_df = pd.DataFrame(Ada_cls_metric_df_list)
Ada_cls_metric_df.to_csv(os.path.join(OUTPUT_DIR, f'Ada_cls_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv'),
                                        index=False)


## Task2: Regression (Hurdle)

### First, without AdaSTEM wrapper

In [13]:
from BirdSTEM.model.AdaSTEM import AdaSTEM, AdaSTEMRegressor, AdaSTEMHurdle
from BirdSTEM.model.Hurdle import Hurdle
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, LinearRegression, BayesianRidge, SGDRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor


In [14]:
reg_model_set_dict = {
    'Hurdle_SGDClassifier_SGDRegressor':Hurdle(classifier=SGDClassifier(random_state=42), 
                                               regressor=SGDRegressor(random_state=42)),
    'Hurdle_Logistic_Linear':Hurdle(classifier=LogisticRegression(random_state=42), regressor=LinearRegression()),
    'Hurdle_SVC_SVR':Hurdle(classifier=SVC(kernel='rbf'), regressor=SVR(kernel='rbf')),
    'Hurdle_DecisionTreeClassifier_DecisionTreeRegressor':Hurdle(classifier=DecisionTreeClassifier(random_state=42),
                                                regressor=DecisionTreeRegressor(random_state=42)),
    'Hurdle_RandomforestClassifier_RandomforestRegressor':Hurdle(classifier=RandomForestClassifier(random_state=42),
                                                regressor=RandomForestRegressor(random_state=42)),
    'Hurdle_MLPClassifier_MLPRegressor': Hurdle(classifier=MLPClassifier(random_state=42),
                                                regressor=MLPRegressor(random_state=42)),
    'Hurdle_XGBClassifier_XGBregressor': Hurdle(classifier=XGBClassifier(tree_method='hist',n_jobs=1), 
                    regressor=XGBRegressor(tree_method='hist',n_jobs=1))
}

In [15]:
K=5

reg_metric_df_list = []
for item in list(reg_model_set_dict.keys())[::-1]:
    model_name = item
    model = reg_model_set_dict[model_name]
    
    kf = KFold(n_splits=K, shuffle=True, random_state=42).split(X, y)
    for kf_count, (train_index, test_index) in tqdm(enumerate(kf), desc=f'{model_name}', total=K):
        X_train = X.iloc[train_index].replace(-1,np.nan)
        imputer = SimpleImputer().fit(X_train[x_names])
        X_train[x_names] = imputer.transform(X_train[x_names])
        scaler = MinMaxScaler().fit(X_train[x_names])
        X_train[x_names] = scaler.transform(X_train[x_names])
        
        y_train = np.where(y[train_index]>0, 1, 0)
        X_test = X.iloc[test_index].replace(-1,np.nan)
        X_test[x_names] = imputer.transform(X_test[x_names])
        X_test[x_names] = scaler.transform(X_test[x_names])
        y_test = np.where(y[test_index]>0, 1, 0)
        
        sample_weights = class_weight.compute_sample_weight(class_weight='balanced',y=np.where(y_train>0,1,0))
        
        
        try:
            start_time = time.time()
            model.fit(X_train[x_names], y_train, sample_weight=sample_weights)
            finish_time = time.time()
            training_time = finish_time - start_time
        except:
            start_time = time.time()
            model.fit(X_train[x_names], y_train)
            finish_time = time.time()
            training_time = finish_time - start_time
            
        start_time = time.time()
        y_pred = model.predict(X_test[x_names])
        y_pred = np.where(y_pred<0, 0, y_pred)
        finish_time = time.time()
        predicting_time = finish_time - start_time
        
        metric_df = AdaSTEM.eval_STEM_res('hurdle', y_test, np.array(y_pred).flatten())
        metric_df['model'] = model_name
        metric_df['task_type'] = 'hurdle'
        metric_df['iter'] = kf_count
        metric_df['sp'] = sp
        metric_df['sample_size'] = SAMPLE_SIZE
        metric_df['training_time'] = training_time
        metric_df['predicting_time'] = predicting_time
        
        reg_metric_df_list.append(metric_df)
        print(metric_df)


    

Hurdle_XGBClassifier_XGBregressor:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.5215024496461621, 'kappa': 0.05870696061431746, 'f1': 0.13636363636363635, 'precision': 0.2727272727272727, 'recall': 0.09090909090909091, 'average_precision': 0.17479338842975206, 'Spearman_r': 0.07001738646266024, 'Pearson_r': 0.07001738646266048, 'R2': -0.3790600270849471, 'MAE': 0.1899999985098839, 'MSE': 0.18999999523162864, 'poisson_deviance_explained': -1.0524139392974612, 'model': 'Hurdle_XGBClassifier_XGBregressor', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.12179875373840332, 'predicting_time': 0.005148887634277344}
{'AUC': 0.5566902080549723, 'kappa': 0.12137310993052719, 'f1': 0.2456140350877193, 'precision': 0.2692307692307692, 'recall': 0.22580645161290322, 'average_precision': 0.1807940446650124, 'Spearman_r': 0.1220115509320933, 'Pearson_r': 0.12201155093209345, 'R2': -0.6415345575500515, 'MAE': 0.2149999964237213, 'MSE': 0.21499998867511796, 'poisson_deviance_explained': -1.068521343709548, 'model': 'Hurdle_XGBC

Hurdle_MLPClassifier_MLPRegressor:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.5370168753402287, 'kappa': 0.08312958435207818, 'f1': 0.2105263157894737, 'precision': 0.25, 'recall': 0.18181818181818182, 'average_precision': 0.18045454545454548, 'Spearman_r': 0.07275893398882982, 'Pearson_r': 0.07611319322640758, 'R2': -0.6485258530139979, 'MAE': 0.22789166247862927, 'MSE': 0.2271256493990036, 'poisson_deviance_explained': -1.1857145328122525, 'model': 'Hurdle_MLPClassifier_MLPRegressor', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 2.663719892501831, 'predicting_time': 0.005728006362915039}
{'AUC': 0.5438060698606604, 'kappa': 0.11894273127753308, 'f1': 0.19047619047619047, 'precision': 0.36363636363636365, 'recall': 0.12903225806451613, 'average_precision': 0.18192082111436952, 'Spearman_r': 0.1432451850855742, 'Pearson_r': 0.14467289887478968, 'R2': -0.2891652070826616, 'MAE': 0.1699103551111154, 'MSE': 0.16884841299765158, 'poisson_deviance_explained': -0.6992782798593224, 'model': 'Hurdle_MLPClassifier_MLP

Hurdle_RandomforestClassifier_RandomforestRegressor:   0%|          | 0/5 [00:00<?, ?it/s]

...

{'AUC': 0.6566866267465069, 'kappa': 0.1978916007791911, 'f1': 0.38596491228070173, 'precision': 0.2716049382716049, 'recall': 0.6666666666666666, 'average_precision': 0.23606995884773663, 'Spearman_r': 0.23695241640037012, 'Pearson_r': 0.2369524164003698, 'R2': -1.5403737978588272, 'MAE': 0.35, 'MSE': 0.35, 'poisson_deviance_explained': -1.0575451845646606, 'model': 'Hurdle_SVC_SVR', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.06715130805969238, 'predicting_time': 0.0372159481048584}
{'AUC': 0.6757014697461348, 'kappa': 0.19922086354290658, 'f1': 0.3833333333333334, 'precision': 0.25842696629213485, 'recall': 0.7419354838709677, 'average_precision': 0.23173613628126136, 'Spearman_r': 0.25590165614489674, 'Pearson_r': 0.2559016561448975, 'R2': -1.824966596678756, 'MAE': 0.37, 'MSE': 0.37, 'poisson_deviance_explained': -1.1206679648006124, 'model': 'Hurdle_SVC_SVR', 'task_type': 'hurdle', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'tr

Hurdle_Logistic_Linear:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.6750136091453457, 'kappa': 0.21130463358527762, 'f1': 0.4, 'precision': 0.27586206896551724, 'recall': 0.7272727272727273, 'average_precision': 0.24562695924764888, 'Spearman_r': 0.26207052220492566, 'Pearson_r': 0.26207052220492627, 'R2': -1.6129559063690793, 'MAE': 0.36, 'MSE': 0.36, 'poisson_deviance_explained': -1.0382740686071994, 'model': 'Hurdle_Logistic_Linear', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.5157120227813721, 'predicting_time': 0.02067875862121582}
{'AUC': 0.6888719221225424, 'kappa': 0.21100330525642386, 'f1': 0.3934426229508197, 'precision': 0.26373626373626374, 'recall': 0.7741935483870968, 'average_precision': 0.23918291386033322, 'Spearman_r': 0.27452863446707226, 'Pearson_r': 0.274528634467073, 'R2': -1.824966596678756, 'MAE': 0.37, 'MSE': 0.37, 'poisson_deviance_explained': -1.094578914678094, 'model': 'Hurdle_Logistic_Linear', 'task_type': 'hurdle', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 't

Hurdle_SGDClassifier_SGDRegressor:   0%|          | 0/5 [00:00<?, ?it/s]

{'AUC': 0.6785519869352206, 'kappa': 0.19166342033502148, 'f1': 0.39416058394160586, 'precision': 0.25961538461538464, 'recall': 0.8181818181818182, 'average_precision': 0.24241258741258745, 'Spearman_r': 0.2663251094697319, 'Pearson_r': 0.2684835157064531, 'R2': -2.109564812123868, 'MAE': 0.4281144706537539, 'MSE': 0.428420291990366, 'poisson_deviance_explained': -1.1447135965306656, 'model': 'Hurdle_SGDClassifier_SGDRegressor', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.0177462100982666, 'predicting_time': 0.013032197952270508}
{'AUC': 0.566711204428326, 'kappa': 0.0469159003959998, 'f1': 0.29702970297029696, 'precision': 0.17543859649122806, 'recall': 0.967741935483871, 'average_precision': 0.1747792869269949, 'Spearman_r': -0.06363098131705121, 'Pearson_r': 0.10058117411042188, 'R2': -4.471377724261052, 'MAE': 0.7189944774368067, 'MSE': 0.7166136974350912, 'poisson_deviance_explained': -1.7058023170438, 'model': 'Hurdle_SGDClassifier_

In [16]:
reg_metric_df = pd.DataFrame(reg_metric_df_list)
reg_metric_df.to_csv(os.path.join(OUTPUT_DIR, f'hurdle_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv'),
                                        index=False)


### Then, with AdaSTEM wrapper

In [17]:
K=5

reg_metric_df_list = []
for item in list(reg_model_set_dict.keys())[::-1]:
    model_name = item
    model = reg_model_set_dict[model_name]

    kf = KFold(n_splits=K, shuffle=True, random_state=42).split(X, y)
    for kf_count, (train_index, test_index) in tqdm(enumerate(kf), desc=f'AdaSTEM + {model_name}', total=K):
        
        try:
            X_train = X.iloc[train_index].replace(-1,np.nan)
            
            new_x_names = list(set(x_names) - set(['DOY']))

            imputer = SimpleImputer().fit(X_train[new_x_names])
            X_train[new_x_names] = imputer.transform(X_train[new_x_names])
            scaler = MinMaxScaler().fit(X_train[new_x_names])
            X_train[new_x_names] = scaler.transform(X_train[new_x_names])

            y_train = y[train_index]
            X_test = X.iloc[test_index].replace(-1,np.nan)
            X_test[new_x_names] = imputer.transform(X_test[new_x_names])
            X_test[new_x_names] = scaler.transform(X_test[new_x_names])
            y_test = y[test_index]

            model = AdaSTEMClassifier(base_model=reg_model_set_dict[model_name], 
                                            sample_weights_for_classifier=True,
                                            ensemble_fold=5,
                                            min_ensemble_require=3,
                                            grid_len_lon_upper_threshold=25,
                                            grid_len_lon_lower_threshold=5,
                                            grid_len_lat_upper_threshold=25,
                                            grid_len_lat_lower_threshold=5,
                                            points_lower_threshold=50,
                                            stixel_training_size_threshold=50,
                                            temporal_start = 1,
                                            temporal_end = 367, 
                                            temporal_step = 30.5, 
                                            temporal_bin_interval = 30.5,
                                            save_tmp=False, 
                                            save_gridding_plot=False)
            
            try:
                start_time = time.time()
                model.fit(X_train[new_x_names + ['DOY','longitude', 'latitude']], y_train)
                finish_time = time.time()
                training_time = finish_time - start_time
                
            except:
                start_time = time.time()
                model.set_params(**{'sample_weights_for_classifier':False})
                model.fit(X_train[new_x_names + ['DOY','longitude', 'latitude']], y_train)
                finish_time = time.time()
                training_time = finish_time - start_time
                
            start_time = time.time()
            y_pred = model.predict(X_test[new_x_names + ['DOY','longitude', 'latitude']])
            y_pred = np.where(y_pred<0, 0, y_pred)
            finish_time = time.time()
            predicting_time_time = finish_time - start_time
                
            metric_df = AdaSTEM.eval_STEM_res('hurdle', y_test, np.array(y_pred).flatten())
            
            metric_df['model'] = 'AdaSTEM_' + model_name
            metric_df['task_type'] = 'hurdle'
            metric_df['iter'] = kf_count
            metric_df['sp'] = sp
            metric_df['sample_size'] = SAMPLE_SIZE
            metric_df['training_time'] = training_time
            metric_df['predicting_time'] = predicting_time
            
            reg_metric_df_list.append(metric_df)
            print(metric_df)
            
        except Exception as e:
            print(e)
            continue


    

AdaSTEM + Hurdle_XGBClassifier_XGBregressor:   0%|          | 0/5 [00:00<?, ?it/s]

training: 100%|██████████| 1191/1191 [00:00<00:00, 8819.93it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.165, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': -0.03706800603393301, 'MAE': 2.025, 'MSE': 114.725, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_Hurdle_XGBClassifier_XGBregressor', 'task_type': 'hurdle', 'iter': 0, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.5594649314880371, 'predicting_time': 0.017216205596923828}


training: 100%|██████████| 1177/1177 [00:00<00:00, 9457.29it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.155, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': -0.06496439224344774, 'MAE': 1.705, 'MSE': 47.655, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_Hurdle_XGBClassifier_XGBregressor', 'task_type': 'hurdle', 'iter': 1, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.3567509651184082, 'predicting_time': 0.017216205596923828}


...

{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.165, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': -0.023765411003576542, 'MAE': 6.64, 'MSE': 1899.29, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_Hurdle_SGDClassifier_SGDRegressor', 'task_type': 'hurdle', 'iter': 2, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.40903663635253906, 'predicting_time': 0.017216205596923828}


training: 100%|██████████| 1136/1136 [00:00<00:00, 9444.87it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.155, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': -0.02458410258839927, 'MAE': 4.105, 'MSE': 702.295, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_Hurdle_SGDClassifier_SGDRegressor', 'task_type': 'hurdle', 'iter': 3, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.349153995513916, 'predicting_time': 0.017216205596923828}


training: 100%|██████████| 1243/1243 [00:00<00:00, 9330.50it/s]


{'AUC': 0.5, 'kappa': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'average_precision': 0.205, 'Spearman_r': nan, 'Pearson_r': nan, 'R2': -0.06920289560797221, 'MAE': 4.23, 'MSE': 276.45, 'poisson_deviance_explained': nan, 'model': 'AdaSTEM_Hurdle_SGDClassifier_SGDRegressor', 'task_type': 'hurdle', 'iter': 4, 'sp': 'Mallard', 'sample_size': 1000, 'training_time': 0.4523739814758301, 'predicting_time': 0.017216205596923828}


In [18]:
reg_metric_df = pd.DataFrame(reg_metric_df_list)
reg_metric_df.to_csv(os.path.join(OUTPUT_DIR, f'Ada_hurdle_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv'),
                                        index=False)


In [19]:
all_metrics = pd.concat([
    pd.read_csv(os.path.join(OUTPUT_DIR, f'cls_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv')),
    pd.read_csv(os.path.join(OUTPUT_DIR, f'Ada_cls_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv')),
    pd.read_csv(os.path.join(OUTPUT_DIR, f'hurdle_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv')),
    pd.read_csv(os.path.join(OUTPUT_DIR, f'Ada_hurdle_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv')),
], axis=0)


In [20]:
all_metrics.to_csv(os.path.join(OUTPUT_DIR, f'ALL_metric_df_SIZE_{SAMPLE_SIZE}_SP_{sp}_year_{year}.csv'))

